# 2D Plate with hole in tension

This simple tutorial show the basic principle to be able to launch mechanical simulation using FEDOO on a very simple example: a plate in tension with hole.


### Import libraries

In [1]:
from fedoo import * #Import all the fedoo library
import numpy as np 
import time #to evaluate the computational time

### Dimension of the problem

The first step is to defined the dimension of the problem. The problem considered here is treated in 2D using the plane stress assumption. We can easily switch to plane strain assumption. 

In [2]:
Util.ProblemDimension("2Dstress") #2D with plane stress assumption
#Util.ProblemDimension("2Dplane") #2D with plane strain assumption


Dimension of the problem is now set on 2Dstress


### Geometry and Mesh
The geometry is imported from the file 'plate_with_hole.msh' using the ImportFromFile method of the Mesh object (static method). 

In [3]:
Mesh.ImportFromFile('plate_with_hole.msh', meshID = "Domain")
#alternative mesh below (uncomment the line)
#Mesh.RectangleMesh(Nx=101, Ny=101, x_min=-50, x_max=50, y_min=-50, y_max=50, ElementShape = 'tri3', ID ="Domain")
type_el = Mesh.GetAll()['Domain'].GetElementShape()
meshID = "Domain"

Mesh imported: "Domain" with elements tri3


### Material definition
For mechanical simulation, a material constitutive law have to be defined associated to an ID.
In this simple problem, an elastic isotropic law named 'ElasticLaw' is defined with:

- Young modulus: $E=100 \times 10^3$ MPa
- Poisson ratio: $\nu = 0.3$

A weak formulation of the internal force problem (mechanical equilibrium equation) is then defined.
The internal force weak formulation without prestress and initial displacement is
$$ \int_\Omega \varepsilon(u^\star) D \varepsilon(u) d\Omega = 0 $$
where $D$ is given by the constitutive law.

By default, the ID of the weak form is the same as the ID of the constitutive law, ie 'ElasticLaw'. 

In [4]:
ConstitutiveLaw.ElasticIsotrop(1e5, 0.3, ID = 'ElasticLaw')
WeakForm.InternalForce("ElasticLaw")

### Matrix assembly
A global matrix assembly is created based on the weakform 'ElasticLaw' and with the ID 'assembling'. The global stiffness matrix is then computed. This matrix can be extracted from the assembly using the function GetMatrix.

In [5]:
Assembly.Create("ElasticLaw", meshID, type_el, ID="Assembling") 
M = Assembly.GetAll()['Assembling'].GetMatrix()

### Definition of the problem and boundary conditions
A static problem is then defined, ie we want to solve: 
$$ K u = F $$
where $K$ is the stiffness matrix computed before and $F$ is the force vector related to external load. 

In [6]:
Problem.Static("Assembling")

<fedoo.libProblem.Problem_Static.Static.<locals>.__Static at 0x14a18288448>

The boundary conditions are included directly to the problem. 
We define firstly two sets of nodes: 
- the left side ("left") 
- the right side ("right")
For that, we use the powerfull numpy function where to find some nodes based on their coordinates. 

In [7]:
#Definition of the set of nodes for boundary conditions
mesh = Mesh.GetAll()[meshID]
crd = mesh.GetNodeCoordinates() 
xmax = np.max(crd[:,0]) ; xmin = np.min(crd[:,0])
mesh.AddSetOfNodes(list(np.where(crd[:,0] == xmin)[0]), "left")
mesh.AddSetOfNodes(list(np.where(crd[:,0] == xmax)[0]), "right")

Application of boundary condition using the BoundaryCondition method. 'Dirichlet' boundary condition (ie displacement boundary conditions) are used. The alternative is 'Neumann' boundary conditions for external force.
It is important to use the method 'ApplyBoundaryCondition()' to include the defined boundary condition in the problem. 
The node 0 is arbitrary bloqued along $\vec{y}$ to avoir rigid body motion.

In [8]:
Problem.BoundaryCondition('Dirichlet','DispX',-5e-1,mesh.GetSetOfNodes("left"))
Problem.BoundaryCondition('Dirichlet','DispX', 5e-1,mesh.GetSetOfNodes("right"))
Problem.BoundaryCondition('Dirichlet','DispY',0,[0])
Problem.ApplyBoundaryCondition()

### Problem resolution
Here the conjugate gradient method is used (solver 'CG')

In [9]:
t0 = time.time() 
Problem.SetSolver('cg')
print('Solving...')
Problem.Solve() 
print('Done in ' +str(time.time()-t0) + ' seconds')

Solving...
Done in 0.0169217586517334 seconds


### Post-treatment
Write a vtk file with:
- The displacement vector
- The strain tensor (nodal and element values)
- The stress tensor (nodal and element values)
- The vonMises stress (nodal and element values)
- The principal Stress (nodal values)
- The 1st and 2nd principal Direction at nodes

Then, print the total amount of stored elastic energy.

In [10]:
#Get the stress tensor (nodal values)
TensorStrain = Assembly.GetAll()['Assembling'].GetStrainTensor(Problem.GetDisp(), "Nodal")       
TensorStress = ConstitutiveLaw.GetAll()['ElasticLaw'].GetStress(TensorStrain)

#Get the stress tensor (element values)
TensorStrainEl = Assembly.GetAll()['Assembling'].GetStrainTensor(Problem.GetDisp(), "Element")       
TensorStressEl = ConstitutiveLaw.GetAll()['ElasticLaw'].GetStress(TensorStrainEl)

# Get the principal directions (vectors on nodes)
PrincipalStress, PrincipalDirection = TensorStress.GetPrincipalStress()

#Get the displacement vector on nodes for export to vtk
U = np.reshape(Problem.GetDoFSolution('all'),(2,-1)).T
N = Mesh.GetAll()[meshID].GetNumberOfNodes()
U = np.c_[U,np.zeros(N)]

#Write the vtk file                            
OUT = Util.ExportData(meshID)

OUT.addNodeData(U,'Displacement')
OUT.addNodeData(TensorStress.vtkFormat(),'Stress')
OUT.addElmData(TensorStressEl.vtkFormat(),'Stress')
OUT.addNodeData(TensorStrain.vtkFormat(),'Strain')
OUT.addElmData(TensorStrainEl.vtkFormat(),'Strain')
OUT.addNodeData(TensorStress.vonMises(),'VMStress')
OUT.addElmData(TensorStressEl.vonMises(),'VMStress')
OUT.addNodeData(PrincipalStress,'PrincipalStress')
OUT.addNodeData(PrincipalDirection[0],'DirPrincipal1')
OUT.addNodeData(PrincipalDirection[1],'DirPrincipal2')

OUT.toVTK("plate_with_hole_in_tension.vtk")
print('Elastic Energy: ' + str(Problem.GetElasticEnergy()))

print('Result file "plate_with_hole_in_tension.vtk" written in the active directory')

Elastic Energy: 35617.10566901006
Result file "plate_with_hole_in_tension.vtk" written in the active directory
